In [2]:
import os 
import functools
import numpy as np
import pandas as pd

# Chapter 2. Vector Autoregressive models
## Least square model example built by hand using numpy
## Original matlab code translated from:
   - Kilian, L., & Lütkepohl, H. (2017). Structural vector autoregressive analysis. Cambridge University Press.
        - pg. 33


- Original matlab files [here](https://sites.google.com/site/lkilian2019/textbook/code)
    - Chapter 2: ls estimate
        - [Download link](https://drive.google.com/file/d/1-aR_PNS_fesCnGjU99o6ifSxE3B0mX-X/view?usp=sharing)
        - Based on ls.m file

## 1. Import data

## 1.A Files

In [3]:
path = 'Data/'
files = [x for x in os.listdir(path) if x.endswith('.txt')]
files.remove('fedfunds.txt')
files

['realgnp.txt', 'gnpdeflator.txt']

## 1.B Import

### 1.B.1 Quarterly data

In [4]:
df = []
for file in files:
    data = pd.read_csv(path + file, delimiter='\t', header = None)
    data_col_name = file.split('.')[0] # Create data column name
    data.columns = ['Year','qtr', data_col_name] # Rename cols
    df.append(data)
#Concat
df = functools.reduce(lambda left,right: pd.merge(left,right,on=['Year','qtr'], how='outer'),df)
df['date'] = pd.to_datetime(df['Year'].astype(str).add('Q').add(df['qtr'].astype(str)))
df['date_qtr'] = df['date'].dt.to_period('Q')
df.set_index('date_qtr', inplace=True) # set index
df.head(2)

,Year,qtr,realgnp,gnpdeflator,date
date_qtr,,,,,
1954Q3,1954,3,2576.3,15.279,1954-07-01
1954Q4,1954,4,2628.2,15.322,1954-10-01


#### 1.B.1.1 Create infl and drgp vars

```Matlab
%Matlab code
infl=dif(log(gnpdeflator(:,3)))*100;
drgdp=dif(log(realgnp(:,3)))*100;
```

In [5]:
df['infl'] = df['gnpdeflator'].apply(np.log).diff(1).multiply(100)
df['drgdp'] = df['realgnp'].apply(np.log).diff(1).multiply(100)
df.head(3)

,Year,qtr,realgnp,gnpdeflator,date,infl,drgdp
date_qtr,,,,,,,
1954Q3,1954,3,2576.3,15.279,1954-07-01,NaN,NaN
1954Q4,1954,4,2628.2,15.322,1954-10-01,0.281037,1.994494
1955Q1,1955,1,2703.1,15.396,1955-01-01,0.481803,2.810006


### 1.B.2 Monthly

In [6]:
df_ff = pd.read_csv(path + 'fedfunds.txt', delimiter='\t', header = None)
df_ff.columns = ['Year', 'month', 'fedfunds']
df_ff['date'] = pd.to_datetime(df_ff['Year'].astype(str).add('-').add(df_ff['month'].astype(str)))
df_ff.set_index('date',inplace=True)
df_ff.head(2)

,Year,month,fedfunds
date,,,
1954-10-01,1954,10,0.85
1954-11-01,1954,11,0.83


#### 1.B.2.1 Convert to irate

```Matlab
%Matlab code
irate=[];
for i=1:3:length(fedfunds(:,3))
  irate=[irate; mean(fedfunds(i:i+2,3))];
end;    
```

In [7]:
df_ff_qtr = df_ff.fedfunds.resample('Q').mean()
df_ff_qtr.index = pd.PeriodIndex(df_ff_qtr.index)
df_ff_qtr = pd.DataFrame(df_ff_qtr)
df_ff_qtr.head(3)

,fedfunds
date,
1954Q4,0.986667
1955Q1,1.343333
1955Q2,1.500000


### 1.B.3 Merge

In [8]:
df = pd.concat([df.drop(['Year','qtr','date', 'realgnp','gnpdeflator'],axis=1), #drop cols
                df_ff_qtr
               ],
          axis=1).dropna()
df = df[['drgdp','fedfunds','infl']]
df.head(5)

,drgdp,fedfunds,infl
1954Q4,1.994494,0.986667,0.281037
1955Q1,2.810006,1.343333,0.481803
1955Q2,1.592811,1.500000,0.414831
1955Q3,1.334583,1.940000,0.708994
1955Q4,0.608906,2.356667,0.977862


## 1.1 VAR lag 4

### 1.1.A Create VAR function

Original matlab code: https://www3.nd.edu/~esims1/olsvarc.m
```Matlab
function [A,SIGMA,U,V] = olsvarc(y,p);
[t,q] = size(y); %t denotes the number of observation on each variable, where there are q total variables
y=y';
Y = y(:,p:t);
for i =1:p-1
    Y = [Y; y(:,p-i:t-i)]; % This forms the companion matrix
end;

X = [ones(1,t-p); Y(:,1:t-p)];
Y=Y(:,2:t-p+1);
A = (Y*X')/(X*X');
U = Y-A*X; %The first two rows of U will be the reduced form residuals
%SIGMA = U*U'/(t-p-p*q-1);
SIGMA = U*U'/(t-p-1);% The upper left hand block gives the variance-covariance matrix
V=A(:,1); %This gives the intercept terms
A=A(:,2:q*p+1);  % This gives the slope coefficients
```


In [9]:
def olsvar(y,p):
    t,q = np.shape(y) #t: num obs each variable, q: total variables
    y = y.transpose()
    Y = y[:, p-1:t+1] # note code is slightly different due to python indexing from 0
    # Create compaion matrix
    for i in range(1, p):
        row_start = p-i-1
        row_end = t-i-1
        #print(row_start, row_end)
        Y = np.concatenate((Y, y[:, row_start: row_end+1]), axis=0)
    x = np.ones((1, t-p))
    x = np.concatenate((x,Y[:, 0:t-p]), axis=0)
    Y =  Y[:, 1:t-p+1+1]
    A_lhs = Y @ x.transpose() #@ is dot product
    A_rhs = x @ x.transpose()
    A = np.dot(A_lhs, np.linalg.pinv(A_rhs)) #A = (Y*X')/(X*X')
    # LS Residuals
    U = Y - A @ x
    sigma = U @ U.transpose() / (t-p-1)
    V = A[:, 0] # intercept terms
    A = A[:, 1:q*p+1] # % This gives the slope coefficients NOTE: slightly diff orig code A=A(:,2:q*p+1);  
    return A, sigma, U, V

### 1.1.B Run VAR: by hand function

In [10]:
y = df.to_numpy()
t,q = np.shape(y) #row, column
p = 4 # 4 lags
A, sigma, U, V = olsvar(y,p)

### 1.1.C Run Var: by statsmodels

In [11]:
from statsmodels.tsa.api import VAR
var = VAR(y).fit(maxlags=4)

### 1.1.1 Intercept terms

#### 1.1.1.1 By hand

In [12]:
vhat = V[0:q].reshape(-1,1)
vhat

array([[ 0.72399638],
       [-0.39248435],
       [ 0.0067436 ]])

#### 1.1.1.2 Statsmodels

In [13]:
var.intercept

array([ 0.72399638, -0.39248435,  0.0067436 ])

### 1.1.2 Innovation co-variance matrix

#### 1.1.3.1 By hand

In [14]:
sighat = sigma[0:q, 0:q]
sighat

array([[ 0.56831732,  0.0749503 , -0.02018028],
       [ 0.0749503 ,  0.61861933,  0.03529739],
       [-0.02018028,  0.03529739,  0.06446127]])

#### 1.1.3.2 Statsmodels

In [15]:
var.sigma_u

array([[ 0.60311226,  0.07953909, -0.02141581],
       [ 0.07953909,  0.65649398,  0.03745846],
       [-0.02141581,  0.03745846,  0.06840787]])

## 1.2 Coefficients

### 1.2.1 ahat matrices contain lag estimate for each equation

In [16]:
# ahat matrices have information for multiple equations
# lag 1 estimates
ahat_1 = A[0:q, 0:q] #A(1:K,1:K)
# lag 2 estimates
ahat_2 = A[0:q, q:2*q] #A(1:K,K+1:2*K)
# lag 3 estimates
ahat_3 = A[0:q, 2*q:3*q] #A(1:K,2*K+1:3*K)
# lag 4 estimates
ahat_4 = A[0:q, 3*q:4*q] #A(1:K,3*K+1:4*K)

In [17]:
for row_num in range(0,3):
    print(f'Equation y{str(row_num+1)}')
    print('Lag 1')
    print(ahat_1[row_num])
    print()

Equation y1
Lag 1
[0.22302261 0.00969635 0.39687534]

Equation y2
Lag 1
[0.31473555 1.09693344 0.59786854]

Equation y3
Lag 1
[0.00122765 0.06355433 0.40961696]



#### 1.1.2.1 Check with statsmodels coefficients
- a matrices are in the coefs matrix in var output

In [18]:
np.allclose(ahat_1, var.coefs[0])
np.allclose(ahat_2, var.coefs[1])
np.allclose(ahat_3, var.coefs[2])
np.allclose(ahat_4, var.coefs[3])

True

## 1.3 Compare with summary

### 1.3.1 Statsmodels

In [19]:
var.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 07, Apr, 2022
Time:                     16:27:07
--------------------------------------------------------------------
No. of Equations:         3.00000    BIC:                   -2.86965
Nobs:                     209.000    HQIC:                  -3.24118
Log likelihood:          -485.620    FPE:                  0.0304139
AIC:                     -3.49334    Det(Omega_mle):       0.0253777
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         0.723996         0.181997            3.978           0.000
L1.y1         0.223023         0.072524            3.075           0.002
L1.y2         0.009696         0.068274            0.142           0.887
L1.y3         0.396875

### 1.3.2 By hand
### Note: y1 data first row of a matrix

#### 1.3.2.1 Results for equation y1

In [20]:
y1_data = pd.DataFrame(A).iloc[0,:].values
y1 = pd.DataFrame(data = y1_data, index =  var.exog_names[1:])
y1

,0
L1.y1,0.223023
L1.y2,0.009696
L1.y3,0.396875
L2.y1,0.214252
L2.y2,-0.386158
L2.y3,0.136048
L3.y1,-0.005326
L3.y2,0.340733
L3.y3,-0.535430
L4.y1,-0.041102


#### 1.3.2.2 Results for all

In [21]:
ys = []
for i in range(0,3):
    data = pd.DataFrame(A).iloc[i,:].values
    y = pd.DataFrame(data = data, index =  var.exog_names[1:])
    col_name = 'eq_' + str(i+1)
    y.columns = [col_name]
    ys.append(y)
pd.concat(ys, axis=1)

,eq_1,eq_2,eq_3
L1.y1,0.223023,0.314736,0.001228
L1.y2,0.009696,1.096933,0.063554
L1.y3,0.396875,0.597869,0.409617
L2.y1,0.214252,0.186689,-0.017425
L2.y2,-0.386158,-0.486032,-0.050985
L2.y3,0.136048,0.503690,0.234997
L3.y1,-0.005326,0.027529,0.011484
L3.y2,0.340733,0.483196,-0.005236
L3.y3,-0.535430,-0.321177,0.081537
L4.y1,-0.041102,-0.022559,0.066688
